In [3]:
import os
from collections import defaultdict
from PIL import Image
from shutil import move

src_dir = 'datasets'
dst_dir = 'filtered_3'

os.makedirs(dst_dir, exist_ok=True)

image_sizes = defaultdict(int)

for filename in os.listdir(src_dir):
    filepath = os.path.join(src_dir, filename)

    if os.path.isfile(filepath) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        try:
            with Image.open(filepath) as img:
                width, height = img.size
                size = (width, height)

                image_sizes[size] += 1
        except PermissionError:
            print(f"Warning: Could not access file {filename}. Skipping.")


# Find max_size
# max_size = max(image_sizes, key=image_sizes.get)

sorted_sizes = sorted(image_sizes.items(), key=lambda x: x[1], reverse=True)

# Get the image size with the third-highest count
third_max_size = sorted_sizes[2][0] if len(sorted_sizes) > 2 else None


for filename in os.listdir(src_dir):
    filepath = os.path.join(src_dir, filename)

    if os.path.isfile(filepath) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        try:
            with Image.open(filepath) as img:
                size = img.size

                if size == third_max_size:
                    move(filepath, os.path.join(dst_dir, filename))
        except PermissionError:
            print(f"Warning: Could not move file {filename}. Skipping.")

print(f'Moved images with size {third_max_size} to {dst_dir}.')


Moved images with size (1600, 1066) to filtered_3.
